# Accidentes de tráfico con implicación de bicicletas

![Accidente Bicicleta](images/acidente-bicicleta.png)

Vamos a utilizar un dataset de accidentes de tráfico de bicicletas que provienen del portal de datos abiertos de Madrid

https://datos.madrid.es/portal/site/egob/menuitem.c05c1f754a33a9fbe4b2e4b284f1a5a0/?vgnextoid=20f4a87ebb65b510VgnVCM1000001d4a900aRCRD&vgnextchannel=374512b9ace9f310VgnVCM100000171f5a0aRCRD&vgnextfmt=default


Una vez preprocesados los datos tendremos un dataset con los siguientes campos:


- Número de expediente del accidente
- Fecha del accidente
- Distrito donde ha ocurrido el accidente
- Tipo de accidente
- Sexo de la persona que ha tenido el accidente
    

Vamos a intentar resolver las siguientes preguntas:

- Datos del número de expediente 2020S000044
- Accidentes que ocurren en el distrito de Moratalaz ordenados por fecha (Descendente)
- Número de accidentes por tipo de accidente (sin ningún orden establecido)
- Número de accidentes cuyo tipo es Colisión frontal

Preprocesado de datos

Se realizará un preprocesado de datos cuyo objetico es limpiar, simplificar y normalizar los datos para que la prática resulte más sencilla

El formato de salida será un fichero Microsoft Excel que contendrá los datos limpios.



## Carga de datos

In [2]:
import pandas as pd

Leemos los datos ...

In [81]:
df = pd.read_csv("data/AccidentesBicicletas_2020.txt", delimiter=";")

## Exploración y limpieza

Limpiamos los datos ...

In [82]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 775 entries, 0 to 774
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   NumExpediente        775 non-null    object 
 1   Fecha                775 non-null    object 
 2   Hora                 775 non-null    object 
 3   Calle                775 non-null    object 
 4   Numero               775 non-null    object 
 5   Distrito             775 non-null    object 
 6   TipoAccidente        775 non-null    object 
 7   EstadoMetereologico  759 non-null    object 
 8   TipoVehiculo         775 non-null    object 
 9   TipoPersona          775 non-null    object 
 10  RangoEdad            775 non-null    object 
 11  Sexo                 763 non-null    object 
 12  Lesividad            684 non-null    float64
dtypes: float64(1), object(12)
memory usage: 78.8+ KB


In [83]:
df = (df
    .assign(Fecha = pd.to_datetime(df.Fecha, format="%d/%m/%Y"),
               Min = pd.to_numeric(df.Hora.str.split(":").str.get(1)),
               Hora = pd.to_numeric(df.Hora.str.split(":").str.get(0))
              )
    .filter(["NumExpediente", "Fecha", "Distrito", "TipoAccidente", "Sexo"])
    # Ponemos las columnas en minúsculas para no se inserten en Postgres con Dobles Comillas
    .rename(columns = {"NumExpediente" : "num_expediente",
                           "Fecha" : "fecha",
                           "Distrito": "distrito",
                           "TipoAccidente" : "tipo_accidente",
                           "Sexo": "sexo"}
              )
)    
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 775 entries, 0 to 774
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   num_expediente  775 non-null    object        
 1   fecha           775 non-null    datetime64[ns]
 2   distrito        775 non-null    object        
 3   tipo_accidente  775 non-null    object        
 4   sexo            763 non-null    object        
dtypes: datetime64[ns](1), object(4)
memory usage: 30.4+ KB


In [84]:
df.sample(10)

,num_expediente,fecha,distrito,tipo_accidente,sexo
734,2020S014646,2020-10-18,TETUÁN,Alcance,Hombre
620,2020S012221,2020-09-08,SAN BLAS-CANILLEJAS,Colisión fronto-lateral,Hombre
654,2020S012877,2020-09-18,CENTRO,Caída,Hombre
625,2020S012398,2020-09-10,RETIRO,Caída,Hombre
531,2020S011015,2020-08-07,CIUDAD LINEAL,Choque contra obstáculo fijo,Hombre
633,2020S012552,2020-09-13,PUENTE DE VALLECAS,Caída,Hombre
26,2020S001078,2020-01-16,MORATALAZ,Colisión fronto-lateral,Hombre
210,2020S007156,2020-05-23,CENTRO,Alcance,Hombre
350,2020S008526,2020-06-20,ARGANZUELA,Caída,Hombre
328,2020S008333,2020-06-16,CHAMARTÍN,Caída,Mujer


Eliminamos los expedientes duplicados / datos nulos para simplificar el ejemplo ...

In [87]:
df = (df
    .drop_duplicates(["num_expediente"])
    .query("sexo.notnull()")
    .reset_index(drop = True)
)   

In [88]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 686 entries, 0 to 685
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   num_expediente  686 non-null    object        
 1   fecha           686 non-null    datetime64[ns]
 2   distrito        686 non-null    object        
 3   tipo_accidente  686 non-null    object        
 4   sexo            686 non-null    object        
dtypes: datetime64[ns](1), object(4)
memory usage: 26.9+ KB


In [90]:
df.head(10)

,num_expediente,fecha,distrito,tipo_accidente,sexo
0,2020S000044,2020-01-02,SAN BLAS-CANILLEJAS,Colisión frontal,Hombre
1,2020S000151,2020-01-04,TETUÁN,Caída,Hombre
2,2020S000212,2020-01-04,FUENCARRAL-EL PARDO,Caída,Hombre
3,2020S003622,2020-01-04,RETIRO,Caída,Mujer
4,2020S000153,2020-01-04,FUENCARRAL-EL PARDO,Colisión múltiple,Hombre
5,2020S000326,2020-01-05,CARABANCHEL,Colisión fronto-lateral,Hombre
6,2020S000996,2020-01-06,LATINA,Colisión fronto-lateral,Hombre
7,2020S000778,2020-01-07,SAN BLAS-CANILLEJAS,Caída,Mujer
8,2020S000350,2020-01-07,RETIRO,Caída,Hombre
9,2020S000305,2020-01-07,RETIRO,Otro,Hombre


## Grabación de los datos limpios

Y finalmente escribimos el resultado a un fichero Excel ,,,

In [91]:
writer = pd.ExcelWriter("data/accidentes.xlsx")
df.to_excel(writer, 'Data', index=False)
writer.save()